<hr style="border:2px solid gray"> </hr>

#  Traveling Salesman Problem

## Example Code

### Algorithm 4: Genetic Algorithm

<hr style="border:2px solid gray"> </hr>

In [1]:
# package list
import numpy as np
import sys
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import time
import random
import functools

# Global Variables
# GA
POOL_SIZE = 20
TOURNAMENT_SIZE = 3
MAX_ITERATION = 20000
TIME_LIMIT = 57
# Plot Settings
PLOT_MODE = True # Draw Route
plt.ion()


In [2]:
#important! 여기는 수정 금지
def timing(func):
    """함수 실행 시간을 측정하는 데코레이터"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} 실행 시간: {end - start:.6f} 초")
        return result
    return wrapper

SEED = 42
np.random.seed(SEED)   # numpy 전역 시드
random.seed(SEED)      # 파이썬 내장 random 시드

In [3]:
file_path = 'round1.txt'

# Step 1. Data Loading

In [4]:
def fileloader():
    #     Data Format
    #     ---------------------------------------------------------
    #     NAME : pia3056
    #     COMMENT : Bonn VLSI data set with 3056 points
    #     COMMENT : Uni Bonn, Research Institute for Discrete Math
    #     COMMENT : Contributed by Andre Rohe
    #     TYPE : TSP
    #     DIMENSION : 3056 -----------------------------|
    #     EDGE_WEIGHT_TYPE : EUC_2D                     |
    #     NODE_COORD_SECTION                            |
    #     1 0 11 (2 dimentional coordinate of city)     |
    #     2 0 115                                       |
    #     ...                                           |
    #     ...(Total 3056 nodes)<------------------------|
    #     EOF
    #     ---------------------------------------------------------
    with open(file_path, "r") as file:
        file_str = file.readlines()

    # Get the coordinates of cities
    coord_str = file_str[8:-1]  # first city string to last city string (EOF 전까지)
    coord_list = np.zeros((len(coord_str), 2))
    for idx, item in enumerate(coord_str):
        items = item.split()
        coord_list[idx, 0], coord_list[idx, 1] = float(items[1]), float(items[2])

    return coord_list

# Step 2. Initialization

In [5]:
def path_cost(path_map, path):

    return path_map[path[:-1], path[1:]].sum()

def initialize_greedy(coord_list, path_map, first_idx):
    n = len(coord_list)
    path = np.zeros(n + 1, dtype=np.int32)
    path[0] = path[-1] = first_idx

    visited = np.ones(n, dtype=bool) # 모두 방문해야 함을 True로 표시
    visited[first_idx] = False # 시작점은 방문한 것으로 처리

    current_city = first_idx
    for i in range(1, n):
        dist = path_map[current_city].copy()
        dist[~visited] = np.inf # 아직 방문하지 않은(True) 도시들 중에서만 선택
        next_city = np.argmin(dist)
        path[i] = next_city
        visited[next_city] = False
        current_city = next_city

    return path

def initialize_random(coord_list, first_idx):
    n = len(coord_list)
    path = np.zeros(n + 1, dtype=np.int32)
    path[0] = path[-1] = first_idx

    rem = np.delete(np.arange(n, dtype=np.int32), first_idx)
    path[1:-1] = np.random.permutation(rem)

    return path

# Step 3. Searching a path

## Algorithm 4. Genetic Algorithm

In [6]:
def initialization(coord_list, path_map, pool_size):
    """초기 해 집단을 생성합니다 (그리디 + 랜덤)."""
    n = len(coord_list)
    path_pool = np.zeros((pool_size, n + 1), dtype=np.int32)
    pool_cost = np.zeros(pool_size)

    # 해 집단의 절반은 그리디, 절반은 랜덤으로 생성
    greedy_count = pool_size // 2

    greedy_starts = np.random.choice(n, greedy_count, replace=False)
    for i in range(greedy_count):
        path_pool[i, :] = initialize_greedy(coord_list, path_map, greedy_starts[i])
        pool_cost[i] = path_cost(path_map, path_pool[i, :])

    for i in range(greedy_count, pool_size):
        start = np.random.randint(0, n)
        path_pool[i, :] = initialize_random(coord_list, start)
        pool_cost[i] = path_cost(path_map, path_pool[i, :])

    return path_pool, pool_cost

In [7]:
def selection(pool_cost, k):
    """토너먼트 선택."""
    selected_indices = np.random.choice(len(pool_cost), k, replace=False)
    tournament_costs = pool_cost[selected_indices]
    winner_local_idx = np.argmin(tournament_costs)
    return selected_indices[winner_local_idx]

def crossover_ox1(p1, p2):
    """Order Crossover (OX1)."""
    n = len(p1) - 1  # 도시의 수

    # 실제 순열이 담긴 경로의 핵심 부분 (길이: n-1)
    p1_core = p1[1:-1]
    p2_core = p2[1:-1]

    # 자식의 핵심 경로 배열 (길이: n-1)
    child_core = np.full(n - 1, -1, dtype=np.int32)

    # 교차점 두 개를 핵심 경로 길이(n-1) 내에서 선택
    start, end = np.sort(np.random.choice(range(n - 1), 2, replace=False))

    # 부모1의 유전자 일부를 자식에게 복사
    child_core[start:end+1] = p1_core[start:end+1]

    # 부모2에서, 자식에게 아직 없는 유전자를 순서대로 추출
    p2_genes_to_add = [gene for gene in p2_core if gene not in child_core]

    # 자식의 비어있는(-1) 공간을 부모2의 유전자로 채움
    p2_idx = 0
    for i in range(n - 1):
        if child_core[i] == -1:
            child_core[i] = p2_genes_to_add[p2_idx]
            p2_idx += 1

    # 전체 경로 재구성 [시작도시, ...핵심경로..., 시작도시]
    final_child = np.zeros(n + 1, dtype=np.int32)
    final_child[0] = final_child[n] = p1[0] # 부모1의 시작 도시를 유지
    final_child[1:-1] = child_core

    return final_child

def mutation_inversion(path):
    """Inversion Mutation."""
    n = len(path) - 1
    # 경로의 핵심 부분(1 ~ n-1) 내에서 두 점 선택
    i, j = np.sort(np.random.choice(range(1, n), 2, replace=False))
    path[i:j+1] = np.flip(path[i:j+1])
    return path

In [8]:
def limited_2_opt(path, path_map, max_attempts=5000):
    """제한된 횟수만큼 무작위 2-opt 교환을 시도하는 지역 탐색."""
    n = len(path) - 1
    best_path = path.copy()
    current_best_cost = path_cost(path_map, best_path)
    for _ in range(max_attempts):
        i, j = np.sort(np.random.choice(range(1, n), 2, replace=False))
        p = best_path
        old_dist = path_map[p[i-1], p[i]] + path_map[p[j], p[j+1]]
        new_dist = path_map[p[i-1], p[j]] + path_map[p[i], p[j+1]]
        if new_dist < old_dist:
            best_path[i:j+1] = np.flip(best_path[i:j+1])
            current_best_cost += (new_dist - old_dist)
    return best_path, current_best_cost

In [9]:
@timing
def memetic_ga(coord_list):
    start_time_main = time.time()
    print('Start Memetic Algorithm (Hybrid GA)')

    path_map = euclidean_distances(coord_list, coord_list)


    MUTATION_RATE = 0.4
    LOCAL_SEARCH_RATE = 0.2

    print('Initialize the population')
    path_pool, pool_cost = initialization(coord_list, path_map, POOL_SIZE)

    best_idx = np.argmin(pool_cost)
    best_path = path_pool[best_idx].copy()
    best_cost = pool_cost[best_idx]

    print(f"Initial best cost: {best_cost:.2f}")

    # --- 시각화 준비 ---
    if PLOT_MODE:
        fig, ax = plt.subplots(figsize=(10, 10))
        plt.ion() # 인터랙티브 모드 켜기

    print('Start the evolution')
    iteration = 0
    while time.time() - start_time_main < TIME_LIMIT:
        iteration += 1

        # ... (선택, 교차, 변이, 지역탐색 등 기존 로직은 동일) ...
        p1_idx = selection(pool_cost, TOURNAMENT_SIZE)
        p2_idx = selection(pool_cost, TOURNAMENT_SIZE)
        child = crossover_ox1(path_pool[p1_idx], path_pool[p2_idx])
        if np.random.rand() < MUTATION_RATE:
            child = mutation_inversion(child)
        child_cost = path_cost(path_map, child)
        if np.random.rand() < LOCAL_SEARCH_RATE:
            child, child_cost = limited_2_opt(child, path_map)

        # ... (교체 로직 동일) ...
        worst_idx = np.argmax(pool_cost)
        if child_cost < pool_cost[worst_idx]:
            path_pool[worst_idx] = child
            pool_cost[worst_idx] = child_cost

        # --- ✨ 시각화 코드 적용 ---
        # 전체 최고 기록 갱신 시 그림을 다시 그림
        if child_cost < best_cost:
            best_cost = child_cost
            best_path = child.copy()
            print(f"New best cost at iter {iteration}: {best_cost:.2f}")

            if PLOT_MODE:
                ax.scatter(coord_list[:, 0], coord_list[:, 1], c='red', s=10)
                ax.set_title(f'City Route: Iteration {iteration} / Cost: {best_cost:.2f}')

                route_xy = coord_list[best_path]
                ax.plot(route_xy[:, 0], route_xy[:, 1], 'k--')

                fig.canvas.draw()
                fig.canvas.flush_events()


    if PLOT_MODE:
        plt.ioff()
        plt.show()

    print(f"Total iterations in {TIME_LIMIT}s: {iteration}")
    return best_path, best_cost

# Main

In [ ]:
try:

    coord_list = fileloader()

    if coord_list is not None:
        num_cities = len(coord_list)
        print(f"Problem loaded with {num_cities} cities.")

        start_time_total = time.time()
        best_path, best_cost = memetic_ga(coord_list)
        end_time_total = time.time()

        print(f"\nTotal Execution Time: {end_time_total - start_time_total:.4f} seconds")
        print(f"Path: {best_path.tolist()}")
        print(f"Final Cost: {best_cost}")

except Exception as e:
    print(f'최종 실행 중 예외 발생: {e}')


Problem loaded with 4475 cities.
Start Memetic Algorithm (Hybrid GA)
Initialize the population
Initial best cost: 16183.40
Start the evolution
